# Table of Contents
 <p><div class="lev1"><a href="#Dynamical-and-Topological-Relationships-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dynamical and Topological Relationships</a></div><div class="lev2"><a href="#Initialize-Environment-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2"><a href="#Generate-List-of-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev2"><a href="#Modules-vs-Subgraphs-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Modules vs Subgraphs</a></div><div class="lev2"><a href="#Inter/Intra-Module-Connectivity-of-Subgraphs-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Inter/Intra Module Connectivity of Subgraphs</a></div><div class="lev2"><a href="#Generate-the-Module-Sensitivity-Index-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Generate the Module-Sensitivity Index</a></div><div class="lev2"><a href="#Temporal-Expression-of-Module-Sensitive-Subgraphs-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Temporal Expression of Module-Sensitive Subgraphs</a></div><div class="lev3"><a href="#Relate-Module-Reconfiguration-to-Subgraph-Reconfiguration-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>Relate Module-Reconfiguration to Subgraph-Reconfiguration</a></div><div class="lev2"><a href="#Subgraph-Topography-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Subgraph Topography</a></div><div class="lev3"><a href="#Relate-Subgraph-Connection-Strength-to-Connection-Distance-1.7.1"><span class="toc-item-num">1.7.1&nbsp;&nbsp;</span>Relate Subgraph Connection Strength to Connection Distance</a></div><div class="lev3"><a href="#Topographical-Sensitivity-1.7.2"><span class="toc-item-num">1.7.2&nbsp;&nbsp;</span>Topographical Sensitivity</a></div><div class="lev3"><a href="#Compute-Subgraph-Dynamics-1.7.3"><span class="toc-item-num">1.7.3&nbsp;&nbsp;</span>Compute Subgraph Dynamics</a></div><div class="lev3"><a href="#Relate-Subgraph-Dynamics-to-Topographical-Sensitivity-1.7.4"><span class="toc-item-num">1.7.4&nbsp;&nbsp;</span>Relate Subgraph Dynamics to Topographical Sensitivity</a></div>

# Dynamical and Topological Relationships

## Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    
except:
    print 'NOT IPYTHON'

from __future__ import division
from IPython.display import display

import os
import sys
import glob
import json

import numpy as np
import pandas as pd
import scipy.stats as stats
import h5py

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import rcParams
import fig_plotting
rcParams = fig_plotting.update_rcparams(rcParams)

os.chdir('../')
import Codebase
os.chdir('./Analysis Notebooks/')

path_CoreData = '/Users/akhambhati/Remotes/CORE.fMRI_multiband.mmattar/restdata'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks'
path_ModData = path_PeriphData + '/e02-DynFuncModule'
path_SubData = path_PeriphData + '/e03-DynFuncSubgraph'
path_ExpData = path_PeriphData + '/e04-CmpModuleSubgraph'

for path in [path_CoreData, path_PeriphData, path_ModData, path_SubData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

## Generate List of Data

In [ ]:
subj_mod_ids = np.unique([full_subj_path.split('/')[-1].split('.')[0]
                          for full_subj_path in glob.iglob('{}/*.consensus_module.npz'.format(path_ModData))])
subj_sub_ids = np.unique([full_subj_path.split('/')[-1].split('.')[0]
                          for full_subj_path in glob.iglob('{}/*.consensus_subgraph.npz'.format(path_SubData))])
subj_cfg_ids = np.unique([full_subj_path.split('/')[-1].split('.')[0]
                          for full_subj_path in glob.iglob('{}/*.cfg_matr.npz'.format(path_SubData))])

if not len(np.intersect1d(subj_mod_ids, subj_sub_ids)) == len(subj_mod_ids):
    raise Exception('Subject files mismatched')
if not len(np.intersect1d(subj_mod_ids, subj_cfg_ids)) == len(subj_mod_ids):
    raise Exception('Subject files mismatched')
    
subj_dict = {}
for subj in subj_mod_ids:
    subj_dict[subj] = {'module_path': '{}/{}.consensus_module.npz'.format(path_ModData, subj),
                       'subgraph_path': '{}/{}.consensus_subgraph.npz'.format(path_SubData, subj),
                       'cfg_path': '{}/{}.cfg_matr.npz'.format(path_SubData, subj)}

## Modules vs Subgraphs

In [ ]:
%matplotlib inline

n_subj = len(subj_dict.keys())
for subj_i, subj in enumerate(['A010615C']): #enumerate(subj_dict.iterkeys()):
    print(" -- Processing: {}".format(subj))
    
    data_cfg = np.load(subj_dict[subj]['cfg_path'], mmap_mode='r')
    data_sub = np.load(subj_dict[subj]['subgraph_path'], mmap_mode='r')
    data_mod = np.load(subj_dict[subj]['module_path'], mmap_mode='r')
    
    
    module_assign = data_mod['module_assign']
    cfg_matr = data_cfg['cfg_matr']
    fac_subgraph = data_sub['fac_subnet']
    fac_coef = data_sub['fac_coef']
        
    n_win = module_assign.shape[0]
    n_node = module_assign.shape[1]
    n_fac = fac_subgraph.shape[0]
    n_conn = fac_subgraph.shape[1]
    
    # Set window index and get modules
    iw = 44
    srt_module = np.argsort(module_assign[iw, :])
    ix, iy = np.meshgrid(srt_module, srt_module)
    max_fac = np.argsort(fac_coef[:, iw])[-1]
    
    # Plot sort subgraph
    plt.figure()
    ax = plt.subplot(111)         
    subg_adj = Codebase.Networks.configuration.convert_conn_vec_to_adj_matr(fac_subgraph[max_fac, :])
    ax.matshow(subg_adj[ix, iy], cmap='rainbow')
    
    # Plot module dividers
    mod_cutoff_ix = np.unique(np.sort(module_assign[iw, :]), return_index=True)[1][1:]
    plt.vlines(mod_cutoff_ix, 0, n_node, color='k', linewidth=5.0)
    plt.hlines(mod_cutoff_ix, 0, n_node, color='k', linewidth=5.0)    

    # Axis Settings
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xlabel('Brain Regions')        
    ax.set_ylabel('Brain Regions')

    #plt.savefig('./e04-Figures/{}-Module_Subgraph-Contrast-TW_{}.svg'.format(subj, iw))
    plt.show()
    plt.close()

## Inter/Intra Module Connectivity of Subgraphs

In [ ]:
%matplotlib inline

fac_coef_pop = []
fac_subgraph_pop = []
n_null = 100

n_subj = len(subj_dict.keys())
for subj_i, subj in enumerate(subj_dict.iterkeys()):
    print(" -- Processing: {}".format(subj))
    
    data_sub = np.load(subj_dict[subj]['subgraph_path'], mmap_mode='r')
    data_mod = np.load(subj_dict[subj]['module_path'], mmap_mode='r')
    
    module_assign = data_mod['module_assign']
    fac_subgraph = data_sub['fac_subnet']
    fac_subgraph_pop.append(fac_subgraph)    
    fac_coef = data_sub['fac_coef']
    fac_coef_pop.append(fac_coef)
        
    n_win = module_assign.shape[0]
    n_node = module_assign.shape[1]
    n_fac = fac_subgraph.shape[0]
    n_conn = fac_subgraph.shape[1]
    
    if subj_i == 0:
        intra_module_conn_pop = np.nan*np.zeros((n_subj, n_fac, n_win))
        inter_module_conn_pop = np.nan*np.zeros((n_subj, n_fac, n_win))
        intra_module_conn_pop_null = np.nan*np.zeros((n_subj, n_null, n_fac, n_win))
        inter_module_conn_pop_null = np.nan*np.zeros((n_subj, n_null, n_fac, n_win))

    
    # Real Intra/Inter Module Subgraph Connectivity
    intra_module_conn = []
    inter_module_conn = []
    for ii in xrange(n_fac):
        norm_subgraph = fac_subgraph[ii, :] / fac_subgraph[ii, :].max()
        subgraph_adj = Codebase.Networks.configuration.convert_conn_vec_to_adj_matr(norm_subgraph)
        
        for iw in xrange(n_win):
            module_id = np.unique(module_assign[iw, :])
            
            if len(module_id) == 1:
                intra_module_conn_pop[subj_i, ii, iw] = 0
                inter_module_conn_pop[subj_i, ii, iw] = 0
            else:            
                for m_id in module_id:
                    in_nodes = np.flatnonzero(module_assign[iw, :] == m_id)
                    out_nodes = np.setdiff1d(np.arange(n_node), in_nodes)

                    in_1, in_2 = np.meshgrid(in_nodes, in_nodes)
                    intra_module_conn.append(np.mean(subgraph_adj[in_1, in_2]))

                    in_1, out_2 = np.meshgrid(in_nodes, out_nodes)                
                    inter_module_conn.append(np.mean(subgraph_adj[in_1, out_2]))
                intra_module_conn_pop[subj_i, ii, iw] = np.mean(intra_module_conn)
                inter_module_conn_pop[subj_i, ii, iw] = np.mean(inter_module_conn)            
    
    # Null Intra/Inter Module Connectivity
    for nn in xrange(n_null):
        intra_module_conn = []
        inter_module_conn = [] 
        for ii in xrange(n_fac):            
            norm_subgraph = fac_subgraph[ii, :] / fac_subgraph[ii, :].max()
            subgraph_adj = Codebase.Networks.configuration.convert_conn_vec_to_adj_matr(norm_subgraph)

            for iw in xrange(n_win):
                null_module_assign = np.random.permutation(module_assign[iw, :])
                module_id = np.unique(null_module_assign)         
            
                if len(module_id) == 1:
                    intra_module_conn_pop_null[subj_i, nn, ii, iw] = 0
                    inter_module_conn_pop_null[subj_i, nn, ii, iw] = 0
                else:            
                    for m_id in module_id:
                        in_nodes = np.flatnonzero(null_module_assign == m_id)
                        out_nodes = np.setdiff1d(np.arange(n_node), in_nodes)

                        in_1, in_2 = np.meshgrid(in_nodes, in_nodes)
                        intra_module_conn.append(np.mean(subgraph_adj[in_1, in_2]))

                        in_1, out_2 = np.meshgrid(in_nodes, out_nodes)
                        inter_module_conn.append(np.mean(subgraph_adj[in_1, out_2]))
                    intra_module_conn_pop_null[subj_i, nn, ii, iw] = np.mean(intra_module_conn)
                    inter_module_conn_pop_null[subj_i, nn, ii, iw] = np.mean(inter_module_conn)      

intra_module_conn_subj = np.nanmean(np.nanmean(intra_module_conn_pop, axis=-1), axis=-1)
inter_module_conn_subj = np.nanmean(np.nanmean(inter_module_conn_pop, axis=-1), axis=-1)
intra_module_conn_subj_null = np.nanmean(np.nanmean(intra_module_conn_pop_null, axis=-1), axis=-1).reshape(-1)
inter_module_conn_subj_null = np.nanmean(np.nanmean(inter_module_conn_pop_null, axis=-1), axis=-1).reshape(-1)
                    
cases_list = [('Intra-Module Subgraph Connectivity', intra_module_conn_subj),
              ('Intra-Module Null Subgraph Connectivity', intra_module_conn_subj_null),
              ('Inter-Module Subgraph Connectivity', inter_module_conn_subj),
              ('Inter-Module Null Subgraph Connectivity', inter_module_conn_subj_null)]

pairs = [[cases_list[0], cases_list[1]],
         [cases_list[0], cases_list[2]],
         [cases_list[2], cases_list[3]],
         [cases_list[1], cases_list[3]]]

for p_i, pair in enumerate(pairs):
    t, p = stats.ttest_ind(pair[0][1], pair[1][1])    
    print('{} - {}: {}, {}'.format(pair[0][0], pair[1][0], t, p))


# Construct result figure
plt.figure()
ax = plt.subplot(111)
bplot = ax.boxplot([cases_list[0][1], cases_list[1][1],
                    cases_list[2][1], cases_list[3][1]],
                   positions=[1, 2, 4, 5], patch_artist=True)
fig_plotting.set_box_color(bplot, 'k', [[0.75, 0.75, 0.75],
                                        [0.25, 0.25, 0.25],
                                        [0.75, 0.75, 0.75],
                                        [0.25, 0.25, 0.25]])
ax.set_xlim([0, 6])

# Axis Settings
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Connection Strength')

#plt.savefig('./e04-Figures/IntraInter_SubgraphModule_Conn.svg'.format(p_i+1))
plt.show()
plt.close()      

## Generate the Module-Sensitivity Index

In [ ]:
# Dimensions: (n_subj, n_fac, n_win)
msi_pop = (intra_module_conn_pop - inter_module_conn_pop) / \
          (intra_module_conn_pop + inter_module_conn_pop)
# Dimensions: (n_subj, n_null, n_fac, n_win)    
msi_pop_null = (intra_module_conn_pop_null - inter_module_conn_pop_null) / \
               (intra_module_conn_pop_null + inter_module_conn_pop_null)
n_subj, n_null, n_fac, n_win = msi_pop_null.shape


msi_avg = np.nanmean(msi_pop, axis=-1)
msi_avg_ord = np.sort(msi_avg, axis=1)
msi_avg_ord_ix = np.argsort(msi_avg, axis=1)

msi_avg_null = np.nanmean(msi_pop_null, axis=-1)
msi_null_dist = np.zeros((3, n_fac))
for fac_ix in xrange(n_fac):
    null_distrib = msi_avg_null[xrange(n_subj), :, msi_avg_ord_ix[:, fac_ix]].reshape(-1)
    msi_null_dist[0, fac_ix] = np.percentile(null_distrib, 25)
    msi_null_dist[1, fac_ix] = np.percentile(null_distrib, 50)
    msi_null_dist[2, fac_ix] = np.percentile(null_distrib, 75)  
    
    print('{}-{}'.format(ii+1, stats.ttest_ind(msi_avg_ord[:, fac_ix],
                                               null_distrib)))
    

plt.figure()
ax = plt.subplot(111)

ax.plot(np.arange(1, 17), msi_null_dist[1, :], color='k', linewidth=1.0)
ax.fill_between(np.arange(1, 17),
                y1=msi_null_dist[0, :],
                y2=msi_null_dist[2, :],
                color=[0.75, 0.75, 0.75])

bplot = ax.boxplot(msi_avg_ord, patch_artist=True)
fig_plotting.set_box_color(bplot, 'k', [[0.0, 0.375, 1.0] for ii in xrange(n_fac)])

# Axis Settings
ax.set_xlim([0, 17])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Module Sensitivity Index');
ax.set_xlabel('Subgraphs');

plt.savefig('./e04-Figures/Module_Sensitivity_Index.svg')                           
plt.show()
plt.close()   
    
    
print(stats.f_oneway(msi_avg_ord[:,0], msi_avg_ord[:,1], msi_avg_ord[:,2], msi_avg_ord[:,3],
                     msi_avg_ord[:,4], msi_avg_ord[:,5], msi_avg_ord[:,6], msi_avg_ord[:,7],
                     msi_avg_ord[:,8], msi_avg_ord[:,9], msi_avg_ord[:,10], msi_avg_ord[:,11],
                     msi_avg_ord[:,12], msi_avg_ord[:,13], msi_avg_ord[:,14], msi_avg_ord[:,15]))

"""
# Compute subgraph/time average measure
module_sensitivity_pop_flat = np.nanmean(np.nanmean(module_sensitivity_pop, axis=-1), axis=-1)
module_sensitivity_pop_null_flat = np.nanmean(np.nanmean(np.nanmean(module_sensitivity_pop_null, axis=-1), axis=-1), axis=-1)

print(stats.ttest_rel(module_sensitivity_pop_flat, module_sensitivity_pop_null_flat))

# Construct result figure
plt.figure()
ax = plt.subplot(111)
bplot = ax.boxplot([module_sensitivity_pop_flat,
                    module_sensitivity_pop_null_flat],
                   positions=[1, 2], patch_artist=True)
fig_plotting.set_box_color(bplot, 'k', [[0.75, 0.75, 0.75],
                                        [0.25, 0.25, 0.25]])

# Axis Settings
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Time-Average\nModule Sensitivity Index')

plt.savefig('./e04-Figures/Module_Sensitivity_Index.svg')                           
plt.show()
plt.close()   
"""

## Temporal Expression of Module-Sensitive Subgraphs

In [ ]:
module_sensitivity_coef = np.zeros((n_subj, n_fac))
module_sensitivity_coef_null = np.zeros((n_subj, n_null, n_fac))

for subj_i in xrange(n_subj):
    for fac_i in xrange(n_fac):
        norm_coef = fac_coef_pop[subj_i][fac_i, :] / fac_coef_pop[subj_i][fac_i, :].max()
        good_tw = np.flatnonzero(~np.isnan(msi_pop[subj_i, fac_i, :]))
        rho, pv = stats.pearsonr(msi_pop[subj_i, fac_i, good_tw], norm_coef[good_tw])
        if np.isnan(rho):
            continue
        module_sensitivity_coef[subj_i, fac_i] = rho
        
        for null_i in xrange(n_null):
            null_coef = np.random.permutation(norm_coef)
            rho, pv = stats.pearsonr(msi_pop[subj_i, fac_i, good_tw], null_coef[good_tw])
            if np.isnan(rho):
                continue
            module_sensitivity_coef_null[subj_i, null_i, fac_i] = rho
        
module_sensitivity_coef_sorted = np.sort(module_sensitivity_coef, axis=1)
module_sensitivity_coef_null_sorted = np.sort(module_sensitivity_coef_null, axis=1).reshape(n_subj*n_null, n_fac)
module_sensitivity_coef_ord = np.argsort(module_sensitivity_coef, axis=1)

# Plot Correlation Between Module-Sensitivity Index and Normalized Temporal Coefficient
plt.figure()
ax = plt.subplot(111)

ax.plot(np.arange(1, 17),
        np.percentile(module_sensitivity_coef_null_sorted, 50, axis=0),
        color='k', linewidth=1.0)
ax.fill_between(np.arange(1, 17),
                y1=np.percentile(module_sensitivity_coef_null_sorted, 25, axis=0),
                y2=np.percentile(module_sensitivity_coef_null_sorted, 75, axis=0),
                color=[0.75, 0.75, 0.75])

for ii in xrange(n_fac):
    print('{}-{}'.format(ii+1, stats.ttest_ind(module_sensitivity_coef_sorted[:, ii],
                                               module_sensitivity_coef_null_sorted[:, ii])))

clr_list = []
for ii in xrange(n_fac):
    clr_list.append([0.0, 0.375, 1.0])
bplot = ax.boxplot(module_sensitivity_coef_sorted, patch_artist=True);
fig_plotting.set_box_color(bplot, 'k', clr_list)

# Axis Settings
ax.set_xlim([0, 17])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Pearson Correlation');
ax.set_xlabel('Subgraphs');

#plt.savefig('./e04-Figures/Module_Sensitivity_Index-TemporalPCorr.svg')
plt.show()
plt.close()              

In [ ]:
s_id = 2
opt_fac = module_sensitivity_coef_ord[s_id,:][-1]

### Plot Example Good and Bad Module sensitivity
plt.figure()
ax = plt.subplot(111)
ax.plot(fac_coef_pop[s_id][opt_fac, :] / fac_coef_pop[s_id][opt_fac, :].max())

# Axis Settings
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Time Windows');
ax.set_ylabel('Expression Coefficient');

ax1 = ax.twinx()
ax1.plot(module_sensitivity_pop[s_id, opt_fac,:] / module_sensitivity_pop[s_id, opt_fac, :].max() )

# Axis Settings
ax1.yaxis.set_ticks_position('right')
ax1.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
ax1.set_ylabel('Module Sensitivity');

#plt.savefig('./e04-Figures/{}-Subgraph_{}-Module_Sensitivity_Index-TemporalCoef.svg'.format(subj_dict.keys()[s_id],
#                                                                                           opt_fac))
plt.show()
plt.close()              

### Relate Module-Reconfiguration to Subgraph-Reconfiguration

In [ ]:
%matplotlib inline               

n_subj = len(subj_dict.keys())
n_null = 1000

red = [240./255, 88./255, 36./255]
blue = [97./255, 153./255, 209./255]

mod_flex_pop = np.zeros(n_subj)
msi_high_skew_pop = np.zeros(n_subj)
msi_low_skew_pop = np.zeros(n_subj)
msi_high_energy_pop = np.zeros(n_subj)
msi_low_energy_pop = np.zeros(n_subj)

for subj_i, subj in enumerate(subj_dict.iterkeys()):
    print(" -- Processing: {}".format(subj))
        
    data_cfg = np.load(subj_dict[subj]['cfg_path'], mmap_mode='r')
    data_sub = np.load(subj_dict[subj]['subgraph_path'], mmap_mode='r')
    data_mod = np.load(subj_dict[subj]['module_path'], mmap_mode='r')
    
    module_assign = data_mod['module_assign']
    cfg_matr = data_cfg['cfg_matr']
    fac_subgraph = data_sub['fac_subnet']
    fac_coef = data_sub['fac_coef']
        
    n_win = module_assign.shape[0]
    n_node = module_assign.shape[1]
    n_fac = fac_subgraph.shape[0]
    n_conn = fac_subgraph.shape[1]
    
    # Module flexibility
    mod_flex_pop[subj_i] = np.mean(np.mean(np.abs(np.diff(module_assign, axis=0)) > 0, axis=0))
    
    # Temporal transience and energy
    fac_coef_norm = fac_coef / fac_coef.max()
    msi_high_ix = np.argsort(np.nanmean(msi_pop[subj_i, ...], axis=-1))[-1]
    msi_low_ix = np.argsort(np.nanmean(msi_pop[subj_i, ...], axis=-1))[0]
    
    msi_high_energy_pop[subj_i] = np.mean(fac_coef_norm[msi_high_ix, :] ** 2)
    msi_low_energy_pop[subj_i] = np.mean(fac_coef_norm[msi_low_ix, :] ** 2)    
    msi_high_skew_pop[subj_i] = stats.skew(fac_coef_norm[msi_high_ix, :]) 
    msi_low_skew_pop[subj_i] = stats.skew(fac_coef_norm[msi_low_ix, :]) 
    

# Relationship to temporal energy
m_high_energy, b_high_energy, rho_high_energy, pv_high_energy, _ = stats.linregress(mod_flex_pop, msi_high_energy_pop)
m_low_energy, b_low_energy, rho_low_energy, pv_low_energy, _ = stats.linregress(mod_flex_pop, msi_low_energy_pop)
print('MSI High: Temporal Energy: r={}, p={}'.format(rho_high_energy, pv_high_energy))
print('MSI Low: Temporal Energy: r={}, p={}'.format(rho_low_energy, pv_low_energy))

plt.figure(); 
ax = plt.subplot(111);
x = np.array([np.min(mod_flex_pop)*0.95, np.max(mod_flex_pop)*1.05])
ax.scatter(mod_flex_pop, msi_low_energy_pop, color=blue, marker='s', alpha=0.6, s=75.0, lw=0)
ax.plot(x, m_low_energy*x + b_low_energy, color=blue)
ax.scatter(mod_flex_pop, msi_high_energy_pop, color=red, marker='o', alpha=0.6, s=75.0, lw=0)
ax.plot(x, m_high_energy*x + b_high_energy, color=red)

# Axis Settings
ax.set_xlim([0.06, 0.12])
ax.set_ylim([0, 0.08])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Subgraph Energy');
ax.set_xlabel('Module Flexibility');

plt.savefig('./e04-Figures/Module_Flexibility-Subgraph_Energy.svg')
plt.show()
plt.close()              

# Relationship to temporal skew
m_high_skew, b_high_skew, rho_high_skew, pv_high_skew, _ = stats.linregress(mod_flex_pop, msi_high_skew_pop)
m_low_skew, b_low_skew, rho_low_skew, pv_low_skew, _ = stats.linregress(mod_flex_pop, msi_low_skew_pop)
print('MSI High: Temporal Skew: r={}, p={}'.format(rho_high_skew, pv_high_skew))
print('MSI Low: Temporal Skew: r={}, p={}'.format(rho_low_skew, pv_low_skew))

plt.figure(); 
ax = plt.subplot(111);
x = np.array([np.min(mod_flex_pop)*0.95, np.max(mod_flex_pop)*1.05])
ax.scatter(mod_flex_pop, msi_low_skew_pop, color=blue, marker='s', alpha=0.6, s=75.0, lw=0)
ax.plot(x, m_low_skew*x + b_low_skew, color=blue)
ax.scatter(mod_flex_pop, msi_high_skew_pop, color=red, marker='o', alpha=0.6, s=75.0, lw=0)
ax.plot(x, m_high_skew*x + b_high_skew, color=red)

# Axis Settings
ax.set_xlim([0.06, 0.12])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Subgraph Transience');
ax.set_xlabel('Module Flexibility');

plt.savefig('./e04-Figures/Module_Flexibility-Subgraph_Skew.svg')
plt.show()
plt.close()              

# Relationship between temporal energy and skew
m_high_skew, b_high_skew, rho_high_skew, pv_high_skew, _ = stats.linregress(msi_high_energy_pop, msi_high_skew_pop)
m_low_skew, b_low_skew, rho_low_skew, pv_low_skew, _ = stats.linregress(msi_low_energy_pop, msi_low_skew_pop)

print('MSI High: Temporal Energy and Skew: r={}, p={}'.format(rho_high_skew, pv_high_skew))
print('MSI Low: Temporal Energy and Skew: r={}, p={}'.format(rho_low_skew, pv_low_skew))

## Subgraph Topography

In [ ]:
# Generate Node Distance Matrix
df = pd.read_csv('{}/Atlas/HOA112_Labels.csv'.format(path_CoreData))
n_node = len(df)

dist_matr = np.zeros((n_node, n_node))
ix, iy = np.mgrid[:n_node, :n_node]

dX = np.array(df.X)
dY = np.array(df.Y)
dZ = np.array(df.Z)

dist_matr[ix, iy] = np.sqrt((dX[ix]-dX[iy])**2 + 
                            (dY[ix]-dY[iy])**2 +
                            (dZ[ix]-dZ[iy])**2)
dist_matr /= dist_matr.max()

plt.figure()        
ax = plt.subplot(111)
mat = ax.matshow(dist_matr, cmap='rainbow')
#plt.colorbar(mat, ax=ax)

# Axis Settings
ax.set_axis_off()
#ax.set_title('Inter-Region Distance')

plt.savefig('./e04-Figures/Topography-InterRegion_Distance.svg')
plt.show()
plt.close()          

### Relate Subgraph Connection Strength to Connection Distance

In [ ]:
%matplotlib inline
n_null = 1000

n_subj = len(subj_dict.keys())
for subj_i, subj in enumerate(subj_dict.iterkeys()):
    print(" -- Processing: {}".format(subj))
    
    data_sub = np.load(subj_dict[subj]['subgraph_path'], mmap_mode='r')
    
    fac_subgraph = data_sub['fac_subnet']
    fac_coef = data_sub['fac_coef']
        
    n_win = module_assign.shape[0]
    n_node = module_assign.shape[1]
    n_fac = fac_subgraph.shape[0]
    n_conn = fac_subgraph.shape[1]

    triu_ix, triu_iy = np.triu_indices(n_node, k=1)
    conn_dist = dist_matr[triu_ix, triu_iy]    
    
    if subj_i == 0:
        topo_sens_pop = np.nan*np.zeros((n_subj, n_fac))
        topo_sens_pop_null = np.nan*np.zeros((n_subj, n_null, n_fac))        

    # Correlation between Subgraph and Distance Matrix
    for f_id in xrange(n_fac):
        fac_subgraph_norm = fac_subgraph[f_id, :] / fac_subgraph[f_id, :].max()
        rho, pval = stats.pearsonr(conn_dist, fac_subgraph_norm)
        topo_sens_pop[subj_i, f_id] = rho
    
    # Null Correlation between Subgraph and Distance Matrix
    for ns in xrange(n_null):
        for f_id in xrange(n_fac):
            fac_subgraph_norm = fac_subgraph[f_id, :] / fac_subgraph[f_id, :].max()        
            rho, pval = stats.pearsonr(conn_dist,
                                    np.random.permutation(fac_subgraph_norm))
            topo_sens_pop_null[subj_i, ns, f_id] = rho

### Topographical Sensitivity

In [ ]:
topo_sens_pop_sorted = np.sort(topo_sens_pop, axis=1)
topo_sens_pop_null_sorted = np.sort(topo_sens_pop_null, axis=1).reshape(n_subj*n_null, n_fac)
topo_sens_pop_ord = np.argsort(topo_sens_pop, axis=1)

for ii in xrange(n_fac):
    print('{}-{}'.format(ii+1, stats.ttest_ind(topo_sens_pop_sorted[:, ii],
                                              topo_sens_pop_null_sorted[:, ii])))

# Plot Topographical Sensitivity
plt.figure()
ax = plt.subplot(111)

ax.plot(np.arange(1, n_fac+1),
        np.percentile(topo_sens_pop_null_sorted, 50, axis=0),
        color='k', linewidth=1.0)
ax.fill_between(np.arange(1, 17),
                y1=np.percentile(topo_sens_pop_null_sorted, 25, axis=0),
                y2=np.percentile(topo_sens_pop_null_sorted, 75, axis=0),
                color=[0.75, 0.75, 0.75])

clr_list = []
for ii in xrange(n_fac):
    clr_list.append([0.0, 0.375, 1.0])
bplot = ax.boxplot(topo_sens_pop_sorted, patch_artist=True);
fig_plotting.set_box_color(bplot, 'k', clr_list)

# Axis Settings
ax.set_xlim([0, 17])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Pearson Correlation');
ax.set_xlabel('Subgraphs');

plt.savefig('./e04-Figures/Topographic_Sensitivity-SubgraphPCorr.svg')
plt.show()
plt.close()              

### Compute Subgraph Dynamics

In [ ]:
tmpr_enrg_pop = np.zeros((n_subj, n_fac))
tmpr_flex_pop = np.zeros((n_subj, n_fac))
tmpr_skew_pop = np.zeros((n_subj, n_fac))

for subj_i in xrange(n_subj):
    for ii, fac_i in enumerate(topo_sens_pop_ord[subj_i, :]):
        fac_subgraph_norm = fac_subgraph_pop[subj_i][fac_i, :] / fac_subgraph_pop[subj_i][fac_i, :].max()
        fac_coef_norm = fac_coef_pop[subj_i][fac_i, :] / fac_coef_pop[subj_i][fac_i, :].max()
                        
        # Temporal energy
        tmpr_enrg = np.mean(fac_coef_norm ** 2)
        tmpr_enrg_pop[subj_i, ii] = tmpr_enrg
        
        # Temporal flexibility
        tmpr_flex = np.mean(np.abs(np.diff(fac_coef_norm)))
        tmpr_flex_pop[subj_i, ii] = tmpr_flex
        
        # Temporal skew
        tmpr_skew = stats.skew(fac_coef_norm)
        tmpr_skew_pop[subj_i, ii] = tmpr_skew        

### Relate Subgraph Dynamics to Topographical Sensitivity

In [ ]:
topo_props = [('Topographic_Sens',   topo_sens_pop_sorted),
              ('Temporal_Energy',    tmpr_enrg_pop),
              ('Temporal_Flex',      tmpr_flex_pop),
              ('Temporal_Skew',      tmpr_skew_pop)]

pairs = []
for topo_1 in topo_props:
    for topo_2 in topo_props:
        
        if (topo_2, topo_1) in pairs:
            continue
            
        if topo_1 == topo_2:
            continue
        
        plt.figure()        
        ax = plt.subplot(111)
                
        ax.scatter(topo_1[1], topo_2[1], color=[0.25, 0.25, 0.25], s=1.0)
        m, b, rho, pval, stderr = stats.linregress(topo_1[1].reshape(-1), topo_2[1].reshape(-1))
        ax.plot(topo_1[1].reshape(-1), m*topo_1[1].reshape(-1)+b, color='k', alpha=0.75)
                
        # Axis Settings
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
        ax.set_xlabel(topo_1[0]);
        ax.set_ylabel(topo_2[0]);

        plt.savefig('./e04-Figures/{}-{}.svg'.format(topo_1[0], topo_2[0]))
        plt.show()
        plt.close()          
        
        print(topo_1[0], topo_2[0], rho, pval)
        
        pairs.append((topo_1, topo_2))